## 1.2 Train Simple Neural Network using Pytorch on MNIST Dataset
- About MNIST Dataset
- Load and Split Dataset Split
- Label Encoding
- Create Simple Neural Network Model
- Run Training Model
- Visualize Training Loss vs Trining Accuracy

### 1.2.1 About MNIST Dataset
- We will use the **MNIST dataset**, a collection of **60,000** labeled handwritten digits dataset in 10 classes.<br>
- Handwritten digits in the MNIST dataset are <i><font color='orange'>28x28 pixel</font></i> grayscale images. 
- The neural network we will build <i><font color='orange'>classifies the handwritten digits</font></i> in their **10 classes** (0, .., 9).
<img src="resource/MNIST.png" width="700px">

In [1]:
import torch
from torchvision import datasets, transforms

In [2]:
!mkdir dataset\train
!mkdir dataset\test

A subdirectory or file dataset\train already exists.
A subdirectory or file dataset\test already exists.


In [ ]:
# Download dataset

# Define data transformations (optional, but recommended)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Download the MNIST dataset
train_set = datasets.MNIST(root='MNIST', train=True, transform=transform, download=True)
test_set = datasets.MNIST(root='MNIST', train=False, transform=transform, download=True)

In [19]:
TRAIN_FOLDER = 'dataset\\train'
TEST_FOLDER = 'dataset\\test'

# Access individual samples
for idx, (image, label) in enumerate(train_set):
    name = f"{TRAIN_FOLDER}\\train_{idx}"
    image_pil = transforms.ToPILImage()(image)
    image_pil.save(f"{name}.png")
    with open(f"{name}.txt", 'w') as label_file:
        label_file.write(str(label))

# Repeat the same process for the test set
for idx, (image, label) in enumerate(test_set):
    name = f"{TEST_FOLDER}\\test_{idx}"
    image_pil = transforms.ToPILImage()(image)
    image_pil.save(f"{name}.png")
    with open(f"{name}.txt", 'w') as label_file:
        label_file.write(str(label))

### 1.2.2 Load & Split Dataset 
- Load each image on MNIST dataset using OpenCV
- <i><font color='orange'>Convert to gray</font></i>, to make sure we only have single channel of 28x28 pixel data on each image
- The simplest approach for classifying them is to use the <i><font color='orange'>28x28=784 pixels</font></i> as inputs for a 1-layer neural network.
- That why we will convert the <i><font color='orange'>2D matrix of 28x28 pixel</font></i> into flatten <i><font color='orange'>1D array 784 pixel</font></i>.<br>
<img src="resource/MNIST_Load.png" width="600px">

In [ ]:
# code load dataset

- Neural Network learned through a training process which requires a <i><font color='orange'>"training dataset"</font></i>. 
- We <i><font color='cyan'>need another dataset, never seen during training</font></i>, to evaluate the "real-world" performance of the network. It is called an <i><font color='orange'>"validation dataset"</font></i>.
- Here we split the 60.000 labeled images MNIST dataset into <i><font color='orange'>60.000 data</font></i> for <i><font color='orange'>"training dataset"</font></i> and <i><font color='orange'>10.000 data</font></i> for <i><font color='orange'>"validation dataset"</font></i> <br>
<img src="resource/MNIST_split.png" width="600px">

In [ ]:
# code split dataset

### 1.2.3 Label Encoding
- Label Encoding help model to map label as <i><font color='orange'>numerical representation</font></i> with ordering.
- Model might learn some <i><font color='orange'>natural ordering between the different class labels</font></i> based on the labels. 
- Assigning them <i><font color='orange'>numbers in a scale</font></i> would implicitly create ordering and relations between different classes.
- For this purpose, we will use <i><font color='orange'>One-hot Encoding</font></i> to encode label of MNIST dataset into look like this,<br>
<img src="resource/MNIS_OneHot.png" width="600px">

In [ ]:
# code apply one hot encoding

### 1.2.4 Create Simple Neural Network Model
- We will creating a simple Neural Network model, with only Input and Output Layer
- The <i><font color='cyan'>Input Layer</font></i> will have <i><font color='orange'>784</font></i> neuron (*same size with flattened 28x28=784 pixels on each MNIST dataset*)
- The <i><font color='cyan'>Output Layer</font></i> will have <i><font color='orange'>10 neuron</font></i> (*same size with a number of dataset class*)<br>
<img src="resource/NN_SingleDense.png" width="600px">


- Each "neuron" in a neural network does a <i><font color='cyan'>weighted sum of all of its inputs</font></i>, adds a constant called the <i><font color='cyan'>"bias"</font></i>.
<img src="resource/NN_03.png" width="400px"><br><br>
- On above network will have <i><font color='cyan'>784x10 weight</font></i>.<br>
<img src="resource/NN_WeightedSum.gif" width="600px"><br><br>
- Then feeds the result through some non-linear <i><font color='cyan'>"activation function"</font></i>. <br>
- We will use <i><font color='cyan'>Softmax</font></i> for that purpose.<br>
<img src="resource/NN_08.png" height="175px"><img src="resource/NN_07.gif" height="175px"><br><br>

In [19]:
# code for neural network model
import torch
import torch.nn as nn

# Define a simple model
model = nn.Sequential(
    nn.Linear(in_features=784, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=10),  # 10 classes (digits 0-9)
    nn.LogSoftmax(dim=1)  # Log probabilities for classification
).to('cuda' if torch.cuda.is_available() else 'cpu')


- Then to <i><font color='cyan'>measure how good</font></i> the trained model, we will measure distance between what the <i><font color='cyan'>network tells us</font></i> and the <i><font color='cyan'>correct answers</font></i>.
- For classification problems we will use <i><font color='cyan'>"cross-entropy distance"</font></i> (a.k.a loss function).
<img src="resource/NN_LOSS.png" width="600px"><br><br>
- <i><font color='cyan'>"Training"</font></i> model actually means using training images and labels to <i><font color='cyan'>adjust weights</font></i> and <i><font color='cyan'>biases</font></i> so as to <i><font color='cyan'>minimise</font></i> the <i><font color='cyan'>cross-entropy</font></i> loss function.
- That process is called <i><font color='cyan'>Optimizer</font></i>. We will talk more about this in <i><font color='cyan'>Pertemuan 2</font></i>, but keep in mind we will use <i><font color='orange'>SGD Optimizer</font></i> for now.<br><br>
- We are also able to add metric to measure like model accuracy, precission, recall, or etc.
- For now we just use <i><font color='orange'>accuracy metric</font></i>.

In [ ]:
# code setup optimizer, loss function & metric

### 1.2.5 Run Training Model
- To run training process, we can use the following code

In [27]:
# code run training

# Training loop
device = torch.device("cpu")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):  # You can adjust the number of epochs
    for images, labels in trainLoader:
        images, labels = images.to('cpu'), labels.to('cpu')
        optimizer.zero_grad()
        output = model(images.view(-1, 784))
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{10}, Loss: {loss.item():.4f}")


Epoch 1/10, Loss: 0.0708
Epoch 2/10, Loss: 0.0123
Epoch 3/10, Loss: 0.0032
Epoch 4/10, Loss: 0.0013
Epoch 5/10, Loss: 0.0018
Epoch 6/10, Loss: 0.0054
Epoch 7/10, Loss: 0.0026
Epoch 8/10, Loss: 0.0155
Epoch 9/10, Loss: 0.0248
Epoch 10/10, Loss: 0.0631


### 1.2.6 Visualize Loss vs Accuracy
- We are also able to visualize Loss vs Accuracy using the following code,
- <i><font color='orange'>Loss will decrease</font></i> as the <i><font color='orange'>epoch increases</font></i>, whereas <i><font color='orange'>accuracy will increase</font></i>.

In [12]:
# code visualize Loss & Accuracy
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testLoader:
        images, labels = images.to(device), labels.to(device)
        output = net(images.view(-1, 784))
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 97.31%


<br><br><br><br><br>
# Source 
- https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist#2